In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **Iteration 기능** **Human-in-the-loop**

`iter()` 메소드는 에이전트의 실행 과정을 단계별로 반복할 수 있도록 해주는 반복자(iterator)를 만듭니다.
<br>
중간 과정에서 사용자의 입력을 받아서 계속 진행할 것인지 물어보는 기능을 제공합니다. <br>
이것을 `Human-in-the-loop` 라고 합니다.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [3]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import tool

도구(tool) 정의

In [4]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

덧셈 계산을 수행하는 Agent 를 정의

In [10]:
tools = [add_function]

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [7]:
# prompt 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [12]:
# Agent 생성
gpt_agent = create_tool_calling_agent(llm, tools, prompt)

In [13]:
# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=False,
    max_iterations=10,
    handle_parsing_errors=True,
)

### **AgentExecutor의 iter()**

AgentExecutor의 실행 과정을 단계별로 반복할 수 있게 해주는 반복자(iterator)를 생성합니다. 
<br><br>

에이전트가 최종 출력에 도달하기까지 거치는 단계들을 순차적으로 접근할 수 있는 AgentExecutorIterator 객체를 반환합니다.
<br><br>

`iter()` 에 질문을 넣게 되면 중간 단계라는 것을 받아올 수 있습니다. <br>
`setp.get()` 으로 결과를 action, value로 구분해서 받을 수 있습니다. <br>
action에서 `action.tool`로 도구(tool)이름을 비교해서 도구를 호출 했었으면 시행결과가 value에 저장됩니다.<br> 
그런다음 사용자가 `input()`를 통해서 계속 진행할 것인지 물어보게 되고 답변에 따라서 계속 진행 또는 멈추게 됩니다.<br>
<br>

`'114.5 + 121.2 + 34.2 + 110.1'` 덧셈 계산을 수행하기 위해서는 단계별로 계산이 수행되게 됩니다. <br>

1. 114.5 + 121.2 = 235.7
2. 235.7 + 34.2 = 270.9
3. 270.9 + 110.1 = 381.0
<br>

이러한 계산 과정을 단계별로 살펴볼 수 있습니다.
<br>

단계별로 계산 결과를 사용자에게 보여주고, 사용자가 계속 진행할지 묻습니다. (Human-in-the-loop) <br>

사용자가 'y'가 아닌 다른 입력을 하면 반복 중단됩니다.

계산할 질문 설정

In [15]:
question = "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"

agent_executor를 반복적으로 실행

In [16]:
for step in agent_executor.iter({"input": question}):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        
        if action.tool == "add_function":            
            print(f"\nTool Name: {action.tool}, 실행 결과: {value}")        # Tool 실행 결과 출력
        
        _continue = input("계속 진행하시겠습니다? (y/n)?:\n") or "Y"        # 사용자에게 계속 진행할지 묻습니다.
        
        if _continue.lower() != "y":                                        # 사용자가 'y'가 아닌 다른 입력을 하면 반복 중단
            break


Tool Name: add_function, 실행 결과: 235.7

Tool Name: add_function, 실행 결과: 380.0


In [17]:
if "output" in step:        
    print(step["output"])

114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.
